In [17]:
# import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [30]:
data = pd.read_csv("temp_matrix (25).csv")
#input variables are columns 0 to 38
#output variables are columps 39 to 41
#the value function makes this a numpy array
X = data.iloc[:,0:38].values
y = data.iloc[:,38:41].values

In [31]:
X

array([[1555390800, '2019-04-16', '9:01:06 AM', ..., nan, nan, 25.9125],
       [1555390800, '2019-04-16', '9:01:06 AM', ..., nan, nan, 25.961],
       [1555394400, '2019-04-16', '10:01:16 AM', ..., nan, nan, 47.0325],
       ...,
       [1567346400, '2019-09-01', '6:07:32 PM', ..., nan, nan, 38.015],
       [1567350000, '2019-09-01', '7:08:16 PM', ..., nan, nan, nan],
       [1567350000, '2019-09-01', '7:08:16 PM', ..., nan, nan, 35.33]],
      dtype=object)

In [21]:
#clean data
clean = data[data.panel=='clean']

In [22]:
#dusty data
dusty = data[data.panel=='dusty']

In [28]:
data.describe()

,timestamp,irradiance,temperature,00,01,02,03,04,05,06,...,24,25,26,27,28,29,30,31,pmax,SCC
count,3.020000e+03,3020.00000,3020.000000,2385.000000,0.0,0.0,2174.000000,0.0,2384.000000,2385.000000,...,0.0,2385.000000,2384.000000,0.0,2385.000000,0.0,0.0,2171.000000,3020.000000,3020.000000
mean,1.561418e+09,491.14655,40.885742,47.141356,NaN,NaN,47.533822,NaN,50.792943,50.576870,...,NaN,49.713923,50.148442,NaN,38.904831,NaN,NaN,45.270815,28575.943819,2820.777815
std,3.462448e+06,384.87222,5.723185,12.900174,NaN,NaN,13.534039,NaN,10.461672,10.421575,...,NaN,10.149967,10.243369,NaN,16.952780,NaN,NaN,12.494428,18972.819979,1831.983514
min,1.555391e+09,0.00000,19.540000,0.000000,NaN,NaN,0.000000,NaN,22.702500,17.644000,...,NaN,22.220000,22.220000,NaN,0.000000,NaN,NaN,0.000000,0.000000,0.000000
25%,1.558501e+09,59.65000,38.300000,38.526000,NaN,NaN,39.194500,NaN,42.637250,42.619000,...,NaN,41.765000,42.428250,NaN,32.782000,NaN,NaN,38.038500,11119.405250,911.000000
50%,1.561451e+09,524.31000,41.470000,48.826000,NaN,NaN,49.637000,NaN,52.454000,52.195000,...,NaN,51.264000,51.682500,NaN,42.721000,NaN,NaN,47.190000,30394.769500,3100.500000
75%,1.564402e+09,855.02000,45.110000,57.077000,NaN,NaN,57.440000,NaN,59.195000,58.870000,...,NaN,57.820000,58.364750,NaN,51.407000,NaN,NaN,53.978000,44546.664000,4396.250000
max,1.567350e+09,1070.61000,53.700000,73.372222,NaN,NaN,79.190000,NaN,72.076250,72.717000,...,NaN,71.750000,72.062000,NaN,68.450000,NaN,NaN,83.440000,72513.456000,6448.000000


In [27]:
data.head()

,timestamp,date,time,day,irradiance,temperature,00,01,02,03,...,25,26,27,28,29,30,31,pmax,SCC,panel
0,1555390800,2019-04-16,9:01:06 AM,Tuesday,557.80,28.21,31.463,NaN,NaN,28.36625,...,26.068,25.936,NaN,26.624,NaN,NaN,25.9125,8801.920,673,clean
1,1555390800,2019-04-16,9:01:06 AM,Tuesday,557.80,28.21,27.705,NaN,NaN,26.07400,...,25.648,25.880,NaN,20.549,NaN,NaN,25.9610,7964.932,630,dusty
2,1555394400,2019-04-16,10:01:16 AM,Tuesday,787.00,33.99,45.556,NaN,NaN,46.98625,...,46.507,47.690,NaN,44.890,NaN,NaN,47.0325,56452.320,4520,clean
3,1555394400,2019-04-16,10:01:16 AM,Tuesday,787.00,33.99,46.665,NaN,NaN,44.95200,...,47.396,48.307,NaN,0.000,NaN,NaN,45.7140,40853.528,4447,dusty
4,1555398000,2019-04-16,11:01:52 AM,Tuesday,916.77,32.31,49.921,NaN,NaN,52.91900,...,51.588,53.025,NaN,49.340,NaN,NaN,52.6700,61858.730,5299,clean


In [55]:
#creating a seperate dataframe
df2=data.loc['date':'day']
df2

,timestamp,date,time,day,irradiance,temperature,00,01,02,03,...,25,26,27,28,29,30,31,pmax,SCC,panel


In [54]:
#missing data : replacing with mean
#strategy options are: mean (using that here), median, and most_frequent 
#axis is the axis you want to perform the action, 0 = column, 1 = row
data.drop(["Team", "Weight"], axis = 1, inplace = True) 
X.interpolate(method='linear', limit_direction='forward', axis=0)
X

TypeError: Cannot interpolate with all object-dtype columns in the DataFrame. Try setting at least one column to a numeric dtype.

In [48]:
data.head()

,timestamp,date,time,day,irradiance,temperature,00,01,02,03,...,25,26,27,28,29,30,31,pmax,SCC,panel
0,1555390800,2019-04-16,9:01:06 AM,Tuesday,557.80,28.21,31.463,NaN,NaN,28.36625,...,26.068,25.936,NaN,26.624,NaN,NaN,25.9125,8801.920,673,clean
1,1555390800,2019-04-16,9:01:06 AM,Tuesday,557.80,28.21,27.705,NaN,NaN,26.07400,...,25.648,25.880,NaN,20.549,NaN,NaN,25.9610,7964.932,630,dusty
2,1555394400,2019-04-16,10:01:16 AM,Tuesday,787.00,33.99,45.556,NaN,NaN,46.98625,...,46.507,47.690,NaN,44.890,NaN,NaN,47.0325,56452.320,4520,clean
3,1555394400,2019-04-16,10:01:16 AM,Tuesday,787.00,33.99,46.665,NaN,NaN,44.95200,...,47.396,48.307,NaN,0.000,NaN,NaN,45.7140,40853.528,4447,dusty
4,1555398000,2019-04-16,11:01:52 AM,Tuesday,916.77,32.31,49.921,NaN,NaN,52.91900,...,51.588,53.025,NaN,49.340,NaN,NaN,52.6700,61858.730,5299,clean
